In [8]:
import numpy as np
from scipy import sparse
np.set_printoptions(linewidth=150)

1. 使用雅可比方法求解稀疏系统，精确到小数点后6位(用无穷范数表示的前向误差)，其中$n=100$和$n=100 000$. 正确解是 $[1,\dots,1]$. 报告所需要的步数和后向误差，系统如下:
$$
\begin{bmatrix}
 3 & -1 \\
-1 &  3 & -1 \\
&\ddots &\ddots &\ddots \\
&&-1&3&-1 \\
&&&-1&3 \\
\end{bmatrix}
\begin{bmatrix}
x_1\\
\\
\vdots\\
\\
x_n
\end{bmatrix}
=
\begin{bmatrix}
2\\
1\\
\vdots\\
1\\
2
\end{bmatrix}
$$

In [36]:
def Jacobi(n, b, toleranceBit, D, DInverse, U, L, answer):
    """
    param: n: Dimension.
    param: D: matrix of principle diagonal.
    param: DInverse: Inverse matrix of D.
    param: U: Matrix with elements above principle diagonal.
    param: L: Matrix with elements below principle diagonal.
    param: b: Constant array.
    """
    x = np.zeros(n)
    tolerance = 10**(-toleranceBit)
    count = 0

    while(np.linalg.norm(x-answer, np.inf)>tolerance and count < 1000):
        count += 1
        x = DInverse@(b-(L+U)@x)

    
    print("Jacobi iteration amount: ", count)
    print("    n: ", n)
    A = D+U+L
    print("   BE: ", np.linalg.norm(A@x-b, np.inf))
    print()
    return x

def Solver(n):
    b = np.ones(n)
    b[0] = 2.
    b[-1] = 2.

    D = sparse.csc_matrix((n, n))
    DInverse = D.copy()
    U = D.copy()
    L = D.copy()

    D.setdiag(3)
    DInverse.setdiag(1/3)
    U.setdiag(-1, k=1)
    L.setdiag(-1, k=-1)

    x = Jacobi(n, b, 6, D, DInverse, U, L, answer=np.ones(n))

Solver(100)
Solver(10000)
Solver(100000)


C:\Users\玛卡巴卡\AppData\Roaming\Python\Python312\site-packages\scipy\sparse\_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Jacobi iteration amount:  35
    n:  100
   BE:  6.867832086365766e-07

Jacobi iteration amount:  35
    n:  10000
   BE:  6.867832086365766e-07

Jacobi iteration amount:  35
    n:  100000
   BE:  6.867832086365766e-07




2. 使用雅可比方法求解稀疏系统，精确到小数点后3位(用无穷范数表示的前向误差)，其中=100. 正确解是 $[1,-1,1,-1,\dots,1,-1]$.报告所需要的步数和后向误差，系统如下:
$$
\begin{bmatrix}
 2 & 1 \\
1 &  2 & 1 \\
&\ddots &\ddots &\ddots \\
&&1&2&1 \\
&&&1&2 \\
\end{bmatrix}
\begin{bmatrix}
x_1\\
\\
\vdots\\
\\
x_n
\end{bmatrix}
=
\begin{bmatrix}
1\\
0\\
\vdots\\
0\\
-1
\end{bmatrix}
$$

3. 重写程序2.2进行高斯-塞德尔迭代，求解例2.24（如下）中的问题验证你的工作.
$$
\begin{bmatrix}
 3&-1& 0& 0& 0&\frac12\\
-1&-3&-1& 0&\frac12& 0\\
 0&-1& 3&-1& 0& 0\\
 0& 0&-1& 3&-1& 0\\
 0&\frac12& 0&-1& 3&-1\\
\frac12& 0& 0& 0&-1& 3\\
\end{bmatrix}
\begin{bmatrix}
u_1\\
u_2\\
u_3\\
u_4\\
u_5\\
u_6\\
\end{bmatrix}
=
\begin{bmatrix}
\frac52\\
\frac32\\
1\\
1\\
\frac32\\
\frac52\\
\end{bmatrix}
\\
解 x=[1,1,1,1,1,1]
$$

4. 重写程序2.2进行SOR.使用 $\omega=1.1$，再次验证例 2.24.


5. 执行编程问题1中的步骤，n=100.
    1. 高斯-塞德尔方法
    2. SOR，$\omega=1.2$.

6. 执行编程问题2中的步骤，
    1. 高斯-塞德尔方法
    2. SOR，$\omega=1.5$.

7. 使用编程问题3的程序，确定形如(2.38)的系统，使用高斯-塞德尔方法在1秒内所能够精确求解的系统最大规模，报告对于不同的n所需的时间，以及前向误差.